### 2021年5月11日 改进CGAN为WCGAN网络,用于生成二分类的数据,这里的数据用的是我们整理出的用电数据

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pandas as pd
import  numpy as np
import random
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

In [2]:
torch.cuda.set_device(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
LEARNING_RATE = 1e-4
BATCH_SIZE=32
IMG_SIZE=67
CHANNELS_IMG=1
NUM_CLASSES = 2
GEN_EMBEDDING = 100
Z_DIM=100
NUM_EPOCHS=500
FEATURES_DISC=16
FEATURES_GEN=16
CRITIC_ITERATIONS =5
LAMBDA_GP = 10

In [4]:
class MnistDataset(Dataset):
    def __init__(self,csv_file):
        self.data_df = pd.read_csv(csv_file,header = 0)
        
        pass
    
    def  __len__(self):
        return len(self.data_df)
    
    def __getitem__(self,index):
        label = self.data_df.iloc[index,0]

        target = torch.zeros((2))
        target[label] = 1.0
        #image_values = torch.FloatTensor(self.data_df.iloc[index,1:].values)/255.0
        image_values = torch.FloatTensor(self.data_df.iloc[index,1:4490].values)/6108.0
        image_values = torch.FloatTensor(image_values).view(1,67,67)
        UserID = self.data_df.index[index]
        
        sample = {'label':label,'image_values':image_values,'target':target,'UserID':UserID}
        #return sample
        return label,image_values,target,UserID,sample
    
    def plot_image(self,index):
        img = self.data_df.iloc[index,1:4490].values.reshape(67,67)
        plt.title("User" + str(self.data_df.index[index])+ "  Label="+ str(self.data_df.iloc[index,0]))
        #plt.title("label = " + str(self.data_df.iloc[index,0]))
        plt.imshow(img,interpolation='none',cmap = 'Blues')
        pass
    pass


In [5]:
mnist_dataset = MnistDataset('./FullDataSet1.csv')
#mnist_dataset.plot_image(1)


In [6]:
# for i in range(4):
#     sample = mnist_dataset[i]
#     print(i,sample['image'].shape,sample['label'],sample['target'],sample['UserID'])
#     ax = plt.subplot(2,2,i+1)
#     plt.tight_layout()
#     plt.imshow((sample['image'].reshape(67,67)),interpolation='none',cmap = 'Blues')
#     plt.title("True Label:" +str(sample['label']) +"  UsrID:" +str(sample['UserID']))
#     #sample.plot_image(i)
#     #plt.imshow(sample[i].image_values.reshape(67,67),cmap = 'gray',interpolation='none')
#     #plt.title("Ground Truth:{}".format(sample[i]))
#     plt.xticks([])
#     plt.yticks([])  

In [7]:
# for label, image_data_tensor, label_tensor in mnist_dataset:
#     print(label)
#     print(label_tensor)

In [8]:
###########这里是为了查看Dataset和DataLoader两个类的作用，mnist_dataset是要处理的数据类，batch_size是每批次需要处理的数据个数
###########这里DataLoader的重点需要看for 循环后的enumerate，即for index,data in enumerate(loader):这里for循环后只有两个变量，
###########index指示该次循环，data表示这一index次抽取出的数据，是个词典，它把所有的数据都拿过来，
###########我们使用data[0],data[1],data[2],data[3],data[4]来表示词典中返回的数值，如原来保存的标签，数值等内容
loader =DataLoader(mnist_dataset,batch_size=BATCH_SIZE,shuffle=True)

In [9]:
# for i,data in enumerate(loader):
#     label, value,target,ID= data[0],data[1],data[2],data[3]
#     print("Index=",i,"Label:",label,"ImageValue:",value," Target=",target,"UserID=",ID)#####这里因为100个用户分批次训练 ，最终剩下4个用户而导致这里的数据有变化

In [10]:
def gradient_penalty(critic,labels,real,fake,device="cpu"):
    BATCH_SIZE,C,H,W = real.shape
    epsilon = torch.rand((BATCH_SIZE,1,1,1)).repeat(1,C,H,W).to(device)
    interpolated_images = real*epsilon + fake*(1-epsilon)
    
    mixed_scores = critic(interpolated_images,labels)
    
    gradient = torch.autograd.grad(
        inputs = interpolated_images,
        outputs = mixed_scores,
        grad_outputs = torch.ones_like(mixed_scores),
        create_graph = True,
        retain_graph = True,      
    )[0]
    
    
    gradient = gradient.view(gradient.shape[0],-1)
    gradient_norm = gradient.norm(2,dim =1)
    gradient_penalty = torch.mean((gradient_norm -1)**2)
    return gradient_penalty

In [11]:
class Discriminator(nn.Module):
    def __init__(self,channels_img,features_d,num_classes,img_size):
        super(Discriminator,self).__init__()
        self.img_size = img_size
        self.disc = nn.Sequential(
                              # input N*channels_img*64*64
            nn.Conv2d(channels_img+1,features_d,kernel_size=4,stride=2,padding=1), #32*32
            nn.LeakyReLU(0.2),
            
            self._block(features_d,features_d*2,4,2,1),#16*16
            self._block(features_d*2,features_d*4,4,2,1),#8*8
            self._block(features_d*4,features_d*8,4,2,1),#4*4
            nn.Conv2d(features_d*8,1,kernel_size=4,stride=2,padding=0), #1*1       
                
        )
        self.embed = nn.Embedding(num_classes,img_size*img_size)
    def _block(self,in_channels,out_channels,kernel_size,stride,padding):
        return nn.Sequential(
        nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size,
        stride,
        padding,
        bias=False,),
        nn.InstanceNorm2d(out_channels,affine=True),
        nn.LeakyReLU(0.2),
        )
    def forward(self,x,labels):
        
        embedding = self.embed(labels).view(labels.shape[0],1,self.img_size,self.img_size)
        x = torch.cat([x,embedding],dim=1)
        return self.disc(x)

In [12]:
class Generator(nn.Module):
    def __init__(self,
                 channels_noise,
                 channels_img,
                 features_g,
                 num_classes,
                 img_size,
                 embed_size,
                ):
        super(Generator,self).__init__()
        self.img_size = img_size
        self.gen = nn.Sequential(
                            # input N*z_dim*1*1
            self._block(channels_noise+embed_size,features_g*16,4,1,0),#N*f_g*16*4*4
            self._block(features_g*16,features_g*8,4,2,1),#8*8
            self._block(features_g*8,features_g*4,4,2,1),#16*16
            self._block(features_g*4,features_g*2,4,2,1),#32*32
            
            nn.ConvTranspose2d(features_g*2,channels_img,kernel_size=5,stride=2,padding=0,) ,#1*1       
            nn.Tanh(),#[-1,1]
        )
        self.embed = nn.Embedding(num_classes,embed_size)
        
    def _block(self,in_channels,out_channels,kernel_size,stride,padding):
        return nn.Sequential(
        nn.ConvTranspose2d(
        in_channels,
        out_channels,
        kernel_size,
        stride,
        padding,
        bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        )
    def forward(self,x,labels):
        #latent vector z:N*noise_dim*1*1
        embedding = self.embed(labels).unsqueeze(2).unsqueeze(3)
        x = torch.cat([x,embedding],dim =1)
        return self.gen(x) 

In [13]:
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m,(nn.Conv2d,nn.ConvTranspose2d,nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data,0.0,0.02)

In [14]:
#dataset = MnistDataset('./FullDataSet1.csv')

In [15]:
# test_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('dataset/',
#                             train=False,download=True,transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
#                             torchvision.transforms.Normalize((0.1307,),(0.3081,))])),batch_size=64,shuffle=True)
# examples= enumerate(test_loader)
# batch_idx,(real,labels) = next(examples)
# fig = plt.figure()
# for i in range(6):
#     plt.subplot(2,3,i+1)
#     plt.tight_layout()
#     plt.imshow(example_data[i][0],cmap = 'gray',interpolation='none')
#     plt.title("Ground Truth:{}".format(example_targets[i]))
#     plt.xticks([])
#     plt.yticks([])

In [16]:
# #################这里是为了测试生成随机标签的Tensor数据，这里需要注意random_idx = random.randint(0,1)，
# #################它的生成范围应该是0-1之间。从这里能够看出，标签为1时，对应的OneHot编码为[0,1],标签为0时，对应的OneHot编码为[1,0],
# label_tensor = torch.zeros((2))
# print(label_tensor)
# random_idx = random.randint(0,1)
# print(random_idx)
# label_tensor[random_idx]=1.0
# print(label_tensor)

In [17]:
gen = Generator(Z_DIM,CHANNELS_IMG,FEATURES_GEN,NUM_CLASSES,IMG_SIZE,GEN_EMBEDDING).to(device)
critic = Discriminator(CHANNELS_IMG,FEATURES_GEN,NUM_CLASSES,IMG_SIZE).to(device)
initialize_weights(gen)
initialize_weights(critic)

In [18]:
opt_gen = optim.Adam(gen.parameters(),lr=LEARNING_RATE,betas=(0.0,0.999))
opt_critic = optim.Adam(critic.parameters(),lr=LEARNING_RATE,betas=(0.0,0.999))

In [19]:
fixed_noise = torch.randn(32,Z_DIM,1,1).to(device)
writer_real = SummaryWriter(f"/home/ncepu/sunmingyu/chenzhiqian/Power-prediction/WGANs/CGAN03/real")
writer_fake = SummaryWriter(f"/home/ncepu/sunmingyu/chenzhiqian/Power-prediction/WGANs/CGAN03/fake")
step = 0

In [20]:
# for batch_idx,data in enumerate(loader):
#     real = data[1]
#     cur_batch_size = real.shape[0]
#     labels = data[0]

In [21]:
# print('real=',real.shape)
# print('cur_batch_size=',cur_batch_size)
# print('labels=',labels,labels.shape)

In [22]:
# noise = torch.randn((cur_batch_size,Z_DIM,1,1))
# print(noise.shape)
# fake = gen(noise,labels)
# print(fake.shape)

In [28]:
gen

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): ConvTranspose2d(200, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): Sequential(
      (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): Sequential(
      (0): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (4): ConvTranspose2d(32, 1, kernel_si

In [29]:
critic

Discriminator(
  (disc): Sequential(
    (0): Conv2d(2, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (5): Conv2d(128, 1, kernel_size=(4, 4), stride=(2, 2))
  )
  (embed): Embedding(2, 4489)

In [23]:
gen.train()
critic.train()

for epoch in range(NUM_EPOCHS):
     
    for batch_idx,data in enumerate(loader):
        real = data[1]
        real = real.to(device)
        cur_batch_size = real.shape[0]
        labels = data[0]
        labels = labels.to(device)
#     for batch_idx, (real,labels) in enumerate(loader):
#         real = real.to(device)
#         cur_batch_size = real.shape[0]
#         labels = labels.to(device)
        
        for _ in range(CRITIC_ITERATIONS):
            noise = torch.randn((cur_batch_size,Z_DIM,1,1)).to(device)
            fake = gen(noise,labels)
            critic_real = critic(real,labels).reshape(-1)
            critic_fake = critic(fake,labels).reshape(-1)
            gp = gradient_penalty(critic,labels,real,fake,device=device)
            loss_critic = ( -(torch.mean(critic_real)-torch.mean(critic_fake)) +LAMBDA_GP*gp)
            critic.zero_grad()
            loss_critic.backward(retain_graph = True)
            opt_critic.step()
     
        
        ###训练生成器 min log(1-D(G(z))) <----->max log(D(G(z))) 
        gen_fake = critic(fake,labels).reshape(-1)
        loss_gen = -torch.mean(gen_fake)        
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()
        
        
        # Print losses osccasionally and print to tensorboard
        if batch_idx %100==0:
            print(
                f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(loader)}\
                Loss D: {loss_critic:.4f},loss G: {loss_gen:.4f}"
            )
            
            with torch.no_grad():
                fake = gen(noise,labels)
                #take out(up to) 32 examples
                img_grid_real = torchvision.utils.make_grid(
                    real[:32],normalize = True
                
                )
                
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32],normalize = True
                
                )
                
                writer_real.add_image("Real",img_grid_real,global_step=step)
                writer_fake.add_image("Fake",img_grid_fake,global_step=step)
                
            step +=1
                   
        
        
    

Epoch [0/500] Batch 0/133                Loss D: -0.0424,loss G: 0.5304
Epoch [0/500] Batch 100/133                Loss D: 0.1810,loss G: -0.1971
Epoch [1/500] Batch 0/133                Loss D: 0.1712,loss G: -0.0703
Epoch [1/500] Batch 100/133                Loss D: 0.2044,loss G: -0.0161
Epoch [2/500] Batch 0/133                Loss D: 0.3919,loss G: -1.0810
Epoch [2/500] Batch 100/133                Loss D: 0.1789,loss G: -2.6590
Epoch [3/500] Batch 0/133                Loss D: 0.2101,loss G: -1.9397
Epoch [3/500] Batch 100/133                Loss D: 0.1410,loss G: -2.3893
Epoch [4/500] Batch 0/133                Loss D: 0.1639,loss G: -1.8506
Epoch [4/500] Batch 100/133                Loss D: 0.1189,loss G: -2.7353
Epoch [5/500] Batch 0/133                Loss D: 0.0943,loss G: -2.7846
Epoch [5/500] Batch 100/133                Loss D: 0.1099,loss G: -2.6172
Epoch [6/500] Batch 0/133                Loss D: 0.0435,loss G: -2.5740
Epoch [6/500] Batch 100/133                Loss D: 0

Epoch [55/500] Batch 100/133                Loss D: 0.0004,loss G: 0.0833
Epoch [56/500] Batch 0/133                Loss D: 0.0049,loss G: -0.1892
Epoch [56/500] Batch 100/133                Loss D: 0.0107,loss G: -0.9765
Epoch [57/500] Batch 0/133                Loss D: 0.0207,loss G: -0.8394
Epoch [57/500] Batch 100/133                Loss D: 0.0040,loss G: -1.3853
Epoch [58/500] Batch 0/133                Loss D: 0.0079,loss G: -1.1532
Epoch [58/500] Batch 100/133                Loss D: 0.0244,loss G: -1.5079
Epoch [59/500] Batch 0/133                Loss D: 0.0258,loss G: -2.0321
Epoch [59/500] Batch 100/133                Loss D: 0.0416,loss G: -1.7675
Epoch [60/500] Batch 0/133                Loss D: 0.0277,loss G: -2.1996
Epoch [60/500] Batch 100/133                Loss D: 0.0207,loss G: -1.1129
Epoch [61/500] Batch 0/133                Loss D: 0.0090,loss G: -0.1629
Epoch [61/500] Batch 100/133                Loss D: 0.0075,loss G: -0.7453
Epoch [62/500] Batch 0/133            

Epoch [111/500] Batch 0/133                Loss D: -0.0050,loss G: -0.8438
Epoch [111/500] Batch 100/133                Loss D: -0.0077,loss G: -0.7112
Epoch [112/500] Batch 0/133                Loss D: -0.0149,loss G: -0.8969
Epoch [112/500] Batch 100/133                Loss D: 0.0333,loss G: -0.4810
Epoch [113/500] Batch 0/133                Loss D: -0.0418,loss G: -1.1372
Epoch [113/500] Batch 100/133                Loss D: 0.0244,loss G: -1.6606
Epoch [114/500] Batch 0/133                Loss D: 0.0002,loss G: -3.1181
Epoch [114/500] Batch 100/133                Loss D: 0.0004,loss G: -3.0210
Epoch [115/500] Batch 0/133                Loss D: -0.0085,loss G: -2.9549
Epoch [115/500] Batch 100/133                Loss D: -0.0129,loss G: -3.4535
Epoch [116/500] Batch 0/133                Loss D: -0.0021,loss G: -4.1700
Epoch [116/500] Batch 100/133                Loss D: -0.0212,loss G: -2.7886
Epoch [117/500] Batch 0/133                Loss D: -0.0214,loss G: -3.1668
Epoch [117/500] B

Epoch [165/500] Batch 100/133                Loss D: -0.0064,loss G: -1.0962
Epoch [166/500] Batch 0/133                Loss D: -0.0033,loss G: -0.7725
Epoch [166/500] Batch 100/133                Loss D: -0.0076,loss G: -0.1869
Epoch [167/500] Batch 0/133                Loss D: -0.0088,loss G: -0.3502
Epoch [167/500] Batch 100/133                Loss D: -0.0047,loss G: -0.1041
Epoch [168/500] Batch 0/133                Loss D: 0.0021,loss G: -0.1583
Epoch [168/500] Batch 100/133                Loss D: -0.0034,loss G: -1.0901
Epoch [169/500] Batch 0/133                Loss D: -0.0048,loss G: -0.7510
Epoch [169/500] Batch 100/133                Loss D: -0.0052,loss G: -0.8453
Epoch [170/500] Batch 0/133                Loss D: -0.0020,loss G: -0.8779
Epoch [170/500] Batch 100/133                Loss D: -0.0031,loss G: -1.0073
Epoch [171/500] Batch 0/133                Loss D: -0.0075,loss G: -1.2807
Epoch [171/500] Batch 100/133                Loss D: -0.0075,loss G: -1.0151
Epoch [172/5

Epoch [220/500] Batch 0/133                Loss D: -0.0106,loss G: -3.6371
Epoch [220/500] Batch 100/133                Loss D: -0.0073,loss G: -3.5918
Epoch [221/500] Batch 0/133                Loss D: 0.0046,loss G: -3.9391
Epoch [221/500] Batch 100/133                Loss D: -0.0100,loss G: -3.1780
Epoch [222/500] Batch 0/133                Loss D: -0.0088,loss G: -3.0397
Epoch [222/500] Batch 100/133                Loss D: -0.0064,loss G: -2.7613
Epoch [223/500] Batch 0/133                Loss D: -0.0041,loss G: -3.0668
Epoch [223/500] Batch 100/133                Loss D: -0.0071,loss G: -2.5560
Epoch [224/500] Batch 0/133                Loss D: -0.0081,loss G: -2.4527
Epoch [224/500] Batch 100/133                Loss D: -0.0067,loss G: -2.0973
Epoch [225/500] Batch 0/133                Loss D: -0.0137,loss G: -2.2754
Epoch [225/500] Batch 100/133                Loss D: -0.0034,loss G: -3.0881
Epoch [226/500] Batch 0/133                Loss D: -0.0078,loss G: -3.0962
Epoch [226/500

Epoch [274/500] Batch 100/133                Loss D: -0.0118,loss G: -2.1343
Epoch [275/500] Batch 0/133                Loss D: -0.0008,loss G: -1.8102
Epoch [275/500] Batch 100/133                Loss D: -0.0161,loss G: -1.7952
Epoch [276/500] Batch 0/133                Loss D: -0.0143,loss G: -1.0900
Epoch [276/500] Batch 100/133                Loss D: -0.0157,loss G: -1.6663
Epoch [277/500] Batch 0/133                Loss D: -0.0129,loss G: -1.4826
Epoch [277/500] Batch 100/133                Loss D: 0.0074,loss G: -3.7842
Epoch [278/500] Batch 0/133                Loss D: -0.0034,loss G: -3.3667
Epoch [278/500] Batch 100/133                Loss D: -0.0024,loss G: -3.9969
Epoch [279/500] Batch 0/133                Loss D: -0.0055,loss G: -4.2577
Epoch [279/500] Batch 100/133                Loss D: -0.0076,loss G: -3.8463
Epoch [280/500] Batch 0/133                Loss D: -0.0070,loss G: -3.7192
Epoch [280/500] Batch 100/133                Loss D: -0.0106,loss G: -3.1218
Epoch [281/5

Epoch [328/500] Batch 100/133                Loss D: -0.0034,loss G: -1.8861
Epoch [329/500] Batch 0/133                Loss D: 0.0056,loss G: -1.9124
Epoch [329/500] Batch 100/133                Loss D: -0.0068,loss G: -2.3652
Epoch [330/500] Batch 0/133                Loss D: -0.0078,loss G: -2.2362
Epoch [330/500] Batch 100/133                Loss D: -0.0166,loss G: -3.2824
Epoch [331/500] Batch 0/133                Loss D: -0.0037,loss G: -3.4914
Epoch [331/500] Batch 100/133                Loss D: -0.0001,loss G: -3.5184
Epoch [332/500] Batch 0/133                Loss D: 0.0012,loss G: -3.6420
Epoch [332/500] Batch 100/133                Loss D: -0.0048,loss G: -3.5053
Epoch [333/500] Batch 0/133                Loss D: -0.0051,loss G: -3.5411
Epoch [333/500] Batch 100/133                Loss D: -0.0078,loss G: -3.1085
Epoch [334/500] Batch 0/133                Loss D: -0.0101,loss G: -3.1504
Epoch [334/500] Batch 100/133                Loss D: 0.0047,loss G: -3.5725
Epoch [335/500

Epoch [382/500] Batch 100/133                Loss D: -0.0095,loss G: -2.5058
Epoch [383/500] Batch 0/133                Loss D: -0.0118,loss G: -2.4363
Epoch [383/500] Batch 100/133                Loss D: -0.0132,loss G: -1.9299
Epoch [384/500] Batch 0/133                Loss D: -0.0139,loss G: -2.4736
Epoch [384/500] Batch 100/133                Loss D: -0.0073,loss G: -2.3368
Epoch [385/500] Batch 0/133                Loss D: -0.0074,loss G: -2.3029
Epoch [385/500] Batch 100/133                Loss D: -0.0107,loss G: -2.6086
Epoch [386/500] Batch 0/133                Loss D: -0.0027,loss G: -2.6790
Epoch [386/500] Batch 100/133                Loss D: -0.0122,loss G: -2.8134
Epoch [387/500] Batch 0/133                Loss D: -0.0174,loss G: -2.6078
Epoch [387/500] Batch 100/133                Loss D: 0.0097,loss G: -2.4047
Epoch [388/500] Batch 0/133                Loss D: -0.0082,loss G: -1.6782
Epoch [388/500] Batch 100/133                Loss D: -0.0153,loss G: -1.8091
Epoch [389/5

Epoch [437/500] Batch 0/133                Loss D: -0.0055,loss G: -0.7065
Epoch [437/500] Batch 100/133                Loss D: -0.0042,loss G: -0.7224
Epoch [438/500] Batch 0/133                Loss D: -0.0049,loss G: -0.4405
Epoch [438/500] Batch 100/133                Loss D: -0.0044,loss G: -0.8453
Epoch [439/500] Batch 0/133                Loss D: -0.0065,loss G: -0.4784
Epoch [439/500] Batch 100/133                Loss D: -0.0046,loss G: 0.4735
Epoch [440/500] Batch 0/133                Loss D: -0.0109,loss G: 0.1718
Epoch [440/500] Batch 100/133                Loss D: -0.0147,loss G: 0.5118
Epoch [441/500] Batch 0/133                Loss D: -0.0132,loss G: -0.2400
Epoch [441/500] Batch 100/133                Loss D: -0.0027,loss G: -0.1991
Epoch [442/500] Batch 0/133                Loss D: -0.0025,loss G: 0.0571
Epoch [442/500] Batch 100/133                Loss D: -0.0049,loss G: -0.0530
Epoch [443/500] Batch 0/133                Loss D: 0.0036,loss G: -0.1672
Epoch [443/500] Ba

Epoch [491/500] Batch 100/133                Loss D: -0.0119,loss G: -0.4647
Epoch [492/500] Batch 0/133                Loss D: -0.0098,loss G: -0.3836
Epoch [492/500] Batch 100/133                Loss D: -0.0056,loss G: -0.2806
Epoch [493/500] Batch 0/133                Loss D: -0.0130,loss G: -0.1735
Epoch [493/500] Batch 100/133                Loss D: -0.0162,loss G: 0.1999
Epoch [494/500] Batch 0/133                Loss D: -0.0206,loss G: 0.1130
Epoch [494/500] Batch 100/133                Loss D: -0.0033,loss G: -3.3836
Epoch [495/500] Batch 0/133                Loss D: -0.0040,loss G: -4.9023
Epoch [495/500] Batch 100/133                Loss D: -0.0180,loss G: -4.6916
Epoch [496/500] Batch 0/133                Loss D: -0.0126,loss G: -3.8746
Epoch [496/500] Batch 100/133                Loss D: -0.0039,loss G: -3.2023
Epoch [497/500] Batch 0/133                Loss D: -0.0053,loss G: -2.7725
Epoch [497/500] Batch 100/133                Loss D: -0.0206,loss G: -3.4384
Epoch [498/50

## 测试生成器的输出

In [24]:
out_noise = torch.randn((100,100,1,1))
out_1 = torch.ones(100)
out_1 = torch.FloatTensor(out_1).long()
out_noise=out_noise.to(device)
out_1 = out_1.to(device)
fake1 = gen(out_noise,out_1).to(device)
print(fake1.shape)
L_1 = fake1.view(100,67*67)*6108
L_1=L_1.cpu().detach().numpy().tolist()
np.savetxt('./L1.csv',L_1,delimiter=',')

torch.Size([100, 1, 67, 67])


In [25]:
out_noise = torch.randn((100,100,1,1))
out_0 = torch.zeros(100)
out_0 = torch.FloatTensor(out_0).long()
out_noise=out_noise.to(device)
out_0 = out_0.to(device)
fake0 = gen(out_noise,out_0).to(device)
print(fake0.shape)
L_0 = fake0.view(100,67*67)*6108
L_0=L_0.cpu().detach().numpy().tolist()
np.savetxt('./L0.csv',L_0,delimiter=',')

torch.Size([100, 1, 67, 67])


In [26]:
batch_out_data = 1000
out_noise = torch.randn((batch_out_data,100,1,1))
out_1 = torch.ones(batch_out_data)
out_1 = torch.FloatTensor(out_1).long()
out_noise=out_noise.to(device)
out_1 = out_1.to(device)
fake1 = gen(out_noise,out_1).to(device)
print(fake1.shape)
L_1 = fake1.view(batch_out_data,67*67)*6108
L_1=L_1.cpu().detach().numpy().tolist()
np.savetxt('./L1000_1.csv',L_1,delimiter=',')

torch.Size([1000, 1, 67, 67])


In [27]:
batch_out_data = 1000
out_noise = torch.randn((batch_out_data,100,1,1))
out_0 = torch.zeros(batch_out_data)
out_0 = torch.FloatTensor(out_0).long()
out_noise=out_noise.to(device)
out_0 = out_0.to(device)
fake0 = gen(out_noise,out_0).to(device)
print(fake0.shape)
L_0 = fake0.view(batch_out_data,67*67)*6108
L_0=L_0.cpu().detach().numpy().tolist()
np.savetxt('./L1000_0.csv',L_0,delimiter=',')

torch.Size([1000, 1, 67, 67])
